In [63]:
import pandas as pd
import numpy as np

import torch

from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoTokenizer

In [64]:
df = pd.read_csv('../data/esnli_train_1.csv') #rmb to train on whole dataset
df.head()

,pairID,gold_label,Sentence1,Sentence2,Explanation_1,WorkerId,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1
0,3416050480.jpg#4r1n,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,the person is not necessarily training his horse,AF0PI3RISB5Q7,A person on a horse jumps over a broken down a...,A person is *training* *his* *horse* for a co...,{},"3,4,5"
1,3416050480.jpg#4r1c,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",One cannot be on a jumping horse cannot be a d...,A36ZT2WFIA2HMF,A person *on* *a* *horse* *jumps* over a brok...,"A person *is* *at* *a* *diner,* *ordering* an...","4,2,3,5","2,5,4,3,6"
2,3416050480.jpg#4r1e,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",a broken down airplane is outdoors,A2GK75ZQTX2RDZ,A person on a horse jumps over *a* *broken* *...,"A person is *outdoors,* on a horse.","8,9,10,7",3
3,2267923837.jpg#2r1n,neutral,Children smiling and waving at camera,They are smiling at their parents,Just because they are smiling and waving at a ...,A18TOIDG32QICP,Children smiling and waving at camera,They are smiling *at* *their* *parents*,{},"5,3,4"
4,2267923837.jpg#2r1e,entailment,Children smiling and waving at camera,There are children present,The children must be present to see them smili...,AEX0YE6TUZRHT,*Children* *smiling* *and* *waving* at camera,There are children *present*,"0,1,3,2",3


In [65]:
def renameColumnsTrain(df):
    return df.rename(columns={'Sentence1': 'premise', 'Sentence2': 'hypothesis', 'Explanation_1': 'explanation'}).drop(["WorkerId", "Sentence1_Highlighted_1", "Sentence2_Highlighted_1"], axis=1)

df_cleaned = renameColumnsTrain(df)

In [66]:
label_to_id = {"entailment": 0, "neutral": 1, "contradiction": 2}
id_to_label = {v: k for k, v in label_to_id.items()}

## Test

In [67]:
# cell for testing the model's output for a single example
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

# Tokenize input
premise = df_cleaned['premise'][0]
hypothesis = df_cleaned['hypothesis'][0]
explanation = df_cleaned['explanation'][0]
actual_label = df_cleaned['gold_label'][0]
encoded_input = tokenizer.encode_plus(premise, hypothesis, explanation, padding=True, truncation=True, return_tensors='pt')

labels = torch.tensor(df_cleaned['gold_label'].replace(label_to_id).tolist())[0]
print(encoded_input)
output = model(**encoded_input)

predicted_class = torch.argmax(output.logits, dim=1)

print(f"Premise: {premise}\nHypothesis: {hypothesis}\nExplanation: {explanation}\n")
print(f"True class: {actual_label}")
print(f"Predicted class: {id_to_label[predicted_class.item()]}")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should pr

{'input_ids': tensor([[    0,   250,   621,    15,    10,  5253, 13855,    81,    10,  3187,
           159, 16847,     4,     2,     2,   250,   621,    16,  1058,    39,
          5253,    13,    10,  1465,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is training his horse for a competition.
Explanation: the person is not necessarily training his horse

True class: neutral
Predicted class: neutral


Concatenate everything together

# RoBERTa classifier

---

In [272]:
def filterNan(df):
    return df.dropna()

# def tokenize(df):
#     return df.apply(lambda x: tokenizer.encode_plus(x['premise'], x['hypothesis'], x['explanation'], padding='max_length', return_tensors='pt'), axis=1)

def convert_to_tensors(df):
    return torch.tensor(df.values)

def encode_labels(df):
    return df.apply(lambda x: label_to_id[x])

template = "Given that {}, it is hypothesized that {}. {}."

def tokenize(df):
    tokenized_batch = []
    for _, row in df.iterrows():
        premise = row['premise'].lower()
        if premise[-1] in ['.', '!', '?']:
            premise = premise[:-1]
        hypothesis = row['hypothesis'].lower()
        if hypothesis[-1] in ['.', '!', '?']:
            hypothesis = hypothesis[:-1]
        explanation = row['explanation'].lower()
        if explanation[-1] in ['.', '!', '?']:
            explanation = explanation[:-1]


        encoded_dict = tokenizer.encode_plus(
            text = template.format(premise, hypothesis, explanation),
            # row['premise'], # two ways to encode
            # row['hypothesis'], 
            # row['explanation'],
            padding=True,
            return_tensors='pt',
            # truncation=True
        )
        tokenized_batch.append(encoded_dict)
    return tokenized_batch

[RoBERTA huggingface](https://huggingface.co/FacebookAI/roberta-base#:~:text=RoBERTa%20is%20a%20transformers%20model%20pretrained%20on%20a,to%20generate%20inputs%20and%20labels%20from%20those%20texts)

In [273]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should pr

In [274]:
# for mac
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# for nvidia GPUs
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [275]:
df_cleaned = renameColumnsTrain(df)
df_cleaned = df_cleaned[:5000] # change accordingly to the size of dataset you want to train on
df_cleaned = filterNan(df_cleaned)
df_cleaned['gold_label'] = encode_labels(df_cleaned['gold_label'])
# split the data into training and validation sets before processing
train_size = int(0.8 * len(df_cleaned))
val_size = len(df_cleaned) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(df_cleaned, [train_size, val_size])
train_dataset = train_dataset.dataset
val_dataset = val_dataset.dataset

tokenized_input_train = tokenize(train_dataset)
tokenized_input_val = tokenize(val_dataset)
train_labels = convert_to_tensors(train_dataset['gold_label'])
val_labels = convert_to_tensors(val_dataset['gold_label'])


# actual_labels = convert_to_tensors(df_cleaned['gold_label'])

In [276]:
tokenized_input[0]

{'input_ids': tensor([[    0, 18377,    14,    10,   621,    15,    10,  5253, 13855,    81,
            10,  3187,   159, 16847,     6,    24,    16, 45936,    14,    10,
           621,    16,  1058,    39,  5253,    13,    10,  1465,     6,   142,
             5,   621,    16,    45,  4784,  1058,    39,  5253,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

## Sample

Template goes like this:

Given that [PREMISE] it is hypothesized that [HYPOTHESIS]. This is <mask> because [EXPLANATION]

if model predicts contradict then,

Given that a person on a horse jumps over a broken down airplane, it is hypothesized that a person is training his horse for a competition. This is <span style="color:red">contradiction</span> because the person is not necessarily training his horse.

In [282]:
mask_token = tokenizer.mask_token
template = "Given that {}, it is hypothesized that {}. This is {} because {}."

In [279]:
sample_premise = df_cleaned['premise'][0].lower()
sample_premise = sample_premise[:-1] if sample_premise[-1] in ['.', '!', '?'] else sample_premise
sample_hypothesis = df_cleaned['hypothesis'][0].lower()
sample_hypothesis = sample_hypothesis[:-1] if sample_hypothesis[-1] in ['.', '!', '?'] else sample_hypothesis
sample_explanation = df_cleaned['explanation'][0].lower()
sample_explanation = sample_explanation[:-1] if sample_explanation[-1] in ['.', '!', '?'] else sample_explanation

In [286]:
encoded_input = tokenizer(template.format(sample_premise, sample_hypothesis, mask_token, sample_explanation),
                          padding="max_length",
                          truncation=True,
                          return_tensors='pt')

In [287]:
tokenizer.batch_decode(encoded_input.input_ids)

['<s>Given that a person on a horse jumps over a broken down airplane, it is hypothesized that a person is training his horse for a competition. This is<mask> because the person is not necessarily training his horse.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [288]:
encoded_input.attention_mask # 1's are for the actual input tokens, 0's are for the padding

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

RoBERTa doesn't have token_type_ids, separate with </s> or tokenizer.sep_token

In [289]:
tokenizer.sep_token

'</s>'

In [290]:
input_ids, attention_mask = encoded_input.values()
output = model(input_ids=input_ids, attention_mask=attention_mask)

In [296]:
pred = output.logits.argmax()
pred, id_to_label[pred.item()], id_to_label[df_cleaned['gold_label'][0]]

(tensor(1), 'neutral', 'neutral')

## Predict without training

### updated

In [297]:
premise_template = 'Given that {}, it is hypothesized that {}.'
explanation_template = ' because {}.'

class eSNLIDataset(Dataset):
    def __init__(self, df, train=True):
        self.df = df
        self.train = train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        example = self.df.iloc[idx,:]
        premise = example["premise"]
        hypothesis = example["hypothesis"]
        explanation = example["explanation"]

        premise = premise_template.format(premise, hypothesis)
        explanation = explanation_template.format(explanation)

        if self.train:
            label = example["gold_label"]
            return premise, explanation, label
        
        return premise, explanation

labels = train_labels

dataset = eSNLIDataset(train_dataset, train=False) # just for testing
dataloader = DataLoader(dataset, batch_size=16)

In [298]:
predictions = []

model.to(device)
model.eval()
with torch.no_grad():
    for batch in dataloader:
        premise, explanation = batch
        encoded_input = tokenizer(premise, explanation, padding=True, truncation=True, return_tensors='pt').to(device)
        outputs = model(**encoded_input)
        logits = outputs.logits

        predictions.extend(logits.argmax(dim=-1).cpu().tolist())



In [300]:
print(calc_f1_score(predictions, labels))

(0.49804746170021025, 0.3316, 0.16601582056673678)


In [301]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# print(tokenized_input)
input_ids = [x['input_ids'].squeeze(0) for x in tokenized_input_train]
input_ids = pad_sequence(input_ids, batch_first=True)
attention_masks = [x['attention_mask'].squeeze(0) for x in tokenized_input_train]
attention_masks = pad_sequence(attention_masks, batch_first=True)

labels = train_labels

dataset = TensorDataset(input_ids, attention_masks, labels)
loader = DataLoader(dataset, batch_size=16)

model.to(device)
model.eval()
predictions = []

with torch.no_grad():
    for batch in loader:
        batch_input_ids, batch_attention_mask, batch_labels = batch

        batch_input_ids = batch_input_ids.to(device)
        batch_attention_mask = batch_attention_mask.to(device)
        
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        logits = outputs.logits.cpu()

        predicted_classes = torch.argmax(logits, dim=1)
        predictions.extend(predicted_classes)

predictions = torch.stack(predictions)

In [302]:
from sklearn.metrics import f1_score

def calc_f1_score(predicted_classes, actual_labels):
    return f1_score(predicted_classes, actual_labels, average='weighted'), f1_score(predicted_classes, actual_labels, average='micro'), f1_score(predicted_classes, actual_labels, average='macro')


In [304]:
print(calc_f1_score(predictions, labels))

(0.49804746170021025, 0.3316, 0.16601582056673678)


Training Loop using Trainer

In [305]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, TensorDataset
import torch

# freeze all the parameters in the base model
for param in model.roberta.parameters():
    param.requires_grad = False

# only train the classification head
for param in model.classifier.parameters():
    param.requires_grad = True

# need to wrap in a dictionary to use the Trainer class
class DictDataset(Dataset):
    def __init__(self, tensor_dataset):
        self.tensor_dataset = tensor_dataset

    def __len__(self):
        return len(self.tensor_dataset)

    def __getitem__(self, idx):
        input_ids, attention_mask, labels = self.tensor_dataset[idx]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

input_ids_train = [x['input_ids'].squeeze(0) for x in tokenized_input_train]
input_ids_train = torch.nn.utils.rnn.pad_sequence(input_ids_train, batch_first=True)
attention_masks_train = [x['attention_mask'].squeeze(0) for x in tokenized_input_train]
attention_masks_train = torch.nn.utils.rnn.pad_sequence(attention_masks_train, batch_first=True)
labels_one_hot_train = torch.nn.functional.one_hot(train_labels, num_classes=3).float()

input_ids_val = [x['input_ids'].squeeze(0) for x in tokenized_input_val]
input_ids_val = torch.nn.utils.rnn.pad_sequence(input_ids_val, batch_first=True)
attention_masks_val = [x['attention_mask'].squeeze(0) for x in tokenized_input_val]
attention_masks_val = torch.nn.utils.rnn.pad_sequence(attention_masks_val, batch_first=True)
labels_one_hot_val = torch.nn.functional.one_hot(val_labels, num_classes=3).float()

tensor_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_one_hot_train)

dataset = DictDataset(tensor_dataset)

validation_tensor_dataset = TensorDataset(input_ids_val, attention_masks_val, labels_one_hot_val)
validation_dataset = DictDataset(validation_tensor_dataset)

training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=10,             
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,                
    evaluation_strategy='steps',     
    eval_steps=50,                   
    save_strategy='epoch',           
    save_steps=100,
)

# Initialize the Trainer with the wrapped dataset
trainer = Trainer(
    model=model,                   
    args=training_args,            
    train_dataset=dataset,
    eval_dataset=validation_dataset
)

# Train
trainer.train()


/Users/dombrr/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3130 [00:00<?, ?it/s]

{'loss': 0.7144, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.03}
{'loss': 0.716, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.06}
{'loss': 0.7095, 'learning_rate': 3e-06, 'epoch': 0.1}
{'loss': 0.7117, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.13}
{'loss': 0.7076, 'learning_rate': 5e-06, 'epoch': 0.16}


  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.7053645253181458, 'eval_runtime': 196.2196, 'eval_samples_per_second': 25.482, 'eval_steps_per_second': 0.403, 'epoch': 0.16}
{'loss': 0.7033, 'learning_rate': 6e-06, 'epoch': 0.19}
{'loss': 0.705, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.22}
{'loss': 0.696, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.26}
{'loss': 0.6886, 'learning_rate': 9e-06, 'epoch': 0.29}
{'loss': 0.6944, 'learning_rate': 1e-05, 'epoch': 0.32}


...

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.6041309833526611, 'eval_runtime': 193.5476, 'eval_samples_per_second': 25.833, 'eval_steps_per_second': 0.408, 'epoch': 9.74}
{'loss': 0.6026, 'learning_rate': 1.3307984790874525e-06, 'epoch': 9.78}
{'loss': 0.5979, 'learning_rate': 1.140684410646388e-06, 'epoch': 9.81}
{'loss': 0.6043, 'learning_rate': 9.505703422053232e-07, 'epoch': 9.84}
{'loss': 0.6009, 'learning_rate': 7.604562737642586e-07, 'epoch': 9.87}
{'loss': 0.5985, 'learning_rate': 5.70342205323194e-07, 'epoch': 9.9}


  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.6041001081466675, 'eval_runtime': 191.7254, 'eval_samples_per_second': 26.079, 'eval_steps_per_second': 0.412, 'epoch': 9.9}
{'loss': 0.6022, 'learning_rate': 3.802281368821293e-07, 'epoch': 9.94}
{'loss': 0.6121, 'learning_rate': 1.9011406844106465e-07, 'epoch': 9.97}
{'loss': 0.6032, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 16153.6749, 'train_samples_per_second': 3.095, 'train_steps_per_second': 0.194, 'train_loss': 0.6225034703081027, 'epoch': 10.0}


TrainOutput(global_step=3130, training_loss=0.6225034703081027, metrics={'train_runtime': 16153.6749, 'train_samples_per_second': 3.095, 'train_steps_per_second': 0.194, 'train_loss': 0.6225034703081027, 'epoch': 10.0})

In [306]:
# predict after training

model.to(device)
model.eval()
predictions = []

with torch.no_grad():
    for batch in validation_dataset:
        batch_input_ids = batch['input_ids'].unsqueeze(0).to(device)
        batch_attention_mask = batch['attention_mask'].unsqueeze(0).to(device)
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        logits = outputs.logits.cpu()

        predicted_classes = torch.argmax(logits, dim=1)
        predictions.extend(predicted_classes)

predictions = torch.stack(predictions)


print(calc_f1_score(predictions, val_labels))

(0.7491518376579546, 0.7461999999999999, 0.7432060968848617)
